In [1]:
!wget https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz
!gunzip numberbatch-en-19.08.txt.gz
!pip install wordfreq
! pip install gensim

--2020-05-09 05:05:48--  https://conceptnet.s3.amazonaws.com/downloads/2019/numberbatch/numberbatch-en-19.08.txt.gz
Resolving conceptnet.s3.amazonaws.com (conceptnet.s3.amazonaws.com)... 52.216.146.123
Connecting to conceptnet.s3.amazonaws.com (conceptnet.s3.amazonaws.com)|52.216.146.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 325403502 (310M) [application/x-gzip]
Saving to: ‘numberbatch-en-19.08.txt.gz’

numberbatch-en-19.0 100%[===================>] 310.33M  16.7MB/s    in 21s     

2020-05-09 05:06:10 (14.8 MB/s) - ‘numberbatch-en-19.08.txt.gz’ saved [325403502/325403502]



In [0]:
import wordfreq
import gensim

2) Load Model


In [3]:
numberbatch = gensim.models.KeyedVectors.load_word2vec_format(
    'numberbatch-en-19.08.txt', 
    binary=False, 
    unicode_errors='ignore'
)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


3) Code to convert to conceptnet style, e.g. word -> /c/en/word

In [0]:
import wordfreq
import re
import numpy as np

In [0]:


# English-specific stopword handling
STOPWORDS = ['the', 'a', 'an']
DROP_FIRST = ['to']
DOUBLE_DIGIT_RE = re.compile(r'[0-9][0-9]')
DIGIT_RE = re.compile(r'[0-9]')

def standardized_uri(language, term):
    """
    Get a URI that is suitable to label a row of a vector space, by making sure
    that both ConceptNet's and word2vec's normalizations are applied to it.

    'language' should be a BCP 47 language code, such as 'en' for English.

    If the term already looks like a ConceptNet URI, it will only have its
    sequences of digits replaced by #. Otherwise, it will be turned into a
    ConceptNet URI in the given language, and then have its sequences of digits
    replaced.
    """
    if not (term.startswith('/') and term.count('/') >= 2):
        term = _standardized_concept_uri(language, term)
    return replace_numbers(term)

def english_filter(tokens):
    """
    Given a list of tokens, remove a small list of English stopwords. This
    helps to work with previous versions of ConceptNet, which often provided
    phrases such as 'an apple' and assumed they would be standardized to
	'apple'.
    """
    non_stopwords = [token for token in tokens if token not in STOPWORDS]
    while non_stopwords and non_stopwords[0] in DROP_FIRST:
        non_stopwords = non_stopwords[1:]
    if non_stopwords:
        return non_stopwords
    else:
        return tokens

def replace_numbers(s):
    """
    Replace digits with # in any term where a sequence of two digits appears.

    This operation is applied to text that passes through word2vec, so we
    should match it.
    """
    if DOUBLE_DIGIT_RE.search(s):
        return DIGIT_RE.sub('#', s)
    else:
        return s

def _standardized_concept_uri(language, term):
    if language == 'en':
        token_filter = english_filter
    else:
        token_filter = None
    language = language.lower()
    norm_text = _standardized_text(term, token_filter)
    return '/c/{}/{}'.format(language, norm_text)

def _standardized_text(text, token_filter):
    tokens = simple_tokenize(text.replace('_', ' '))
    if token_filter is not None:
        tokens = token_filter(tokens)
    return '_'.join(tokens)

def simple_tokenize(text):
    """
    Tokenize text using the default wordfreq rules.
    """
    return wordfreq.tokenize(text, 'xx')

In [5]:
res = standardized_uri('en', 'dog')
print(standardized_uri('en', 'dog'))

/c/en/dog


In [6]:
numberbatch.vectors

array([[ 0.0295, -0.0405, -0.0341, ..., -0.0239,  0.0403, -0.0005],
       [ 0.0202, -0.0249, -0.0653, ..., -0.0575, -0.058 ,  0.0651],
       [ 0.0521, -0.0262, -0.0881, ..., -0.0424, -0.0192,  0.0599],
       ...,
       [-0.0356,  0.0516, -0.1663, ..., -0.0009, -0.0191, -0.0744],
       [ 0.0455, -0.0179, -0.0083, ...,  0.045 ,  0.0528,  0.0087],
       [-0.1069, -0.0508,  0.0503, ..., -0.0099, -0.0186, -0.005 ]],
      dtype=float32)

In [11]:
#embedding_matrix = sparse.lil_matrix((len(tok.word_index) + 1, vec_len))
for pair in zip(numberbatch.wv.index2word, numberbatch.wv.syn0):
  # add the word if the word in the tokenizer
  if pair[0] in ['boy','car','other']:
    print(pair)
    #embedding_matrix[tok.word_index[pair[0]]] = pair[1]
  # save the matrix to the dir
#np.save(pair[1]+'.npy', embedding_matrix.toarray()) 

('boy', array([ 0.0194, -0.0464,  0.0931, -0.0062, -0.1372,  0.0674, -0.0597,
       -0.0468,  0.0528,  0.0725, -0.033 ,  0.0186, -0.08  , -0.1483,
       -0.0197, -0.0277, -0.0074,  0.0682,  0.0124,  0.0493,  0.0696,
       -0.0873,  0.0308, -0.0574, -0.0363, -0.0498, -0.0678,  0.0366,
       -0.0761,  0.0057,  0.0172, -0.0418, -0.005 ,  0.0552, -0.0335,
        0.0035, -0.0655, -0.0011, -0.1268,  0.0112, -0.0621,  0.1556,
       -0.0118, -0.108 ,  0.0572,  0.0081,  0.0409,  0.0792, -0.056 ,
        0.0449, -0.0798, -0.02  , -0.0328, -0.1329, -0.1324,  0.0852,
       -0.0167,  0.0213, -0.1082,  0.0194, -0.1177,  0.089 ,  0.0198,
        0.005 , -0.0471, -0.0557, -0.0086, -0.0759,  0.0878, -0.0414,
        0.0152, -0.0296,  0.0411, -0.0019,  0.126 , -0.1049, -0.0466,
        0.0242,  0.0438, -0.0039,  0.0078, -0.0189,  0.0869, -0.0235,
       -0.0932,  0.0921, -0.0188,  0.0718, -0.0539,  0.0149, -0.1129,
        0.0594,  0.0643, -0.0109,  0.0027, -0.1022,  0.0158,  0.0273,
       -0.06

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  """Entry point for launching an IPython kernel.
